In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string

In [2]:
full_df = pd.read_csv('/content/1429_1.csv', nrows=5000)
full_df.head()

,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,NaN,DaveZ
3,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5620406/review...,I've had my Fire HD 8 two weeks now and I love...,Good!!!,NaN,NaN,Shacks
4,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,NaN,NaN,explore42


In [3]:
df = full_df[["reviews.text","reviews.title"]]
df["reviews.text"] = df["reviews.text"].astype(str)
df["reviews.title"]=df["reviews.title"].astype(str)

<ipython-input-3-cec0b0626797>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["reviews.text"] = df["reviews.text"].astype(str)
<ipython-input-3-cec0b0626797>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["reviews.title"]=df["reviews.title"].astype(str)


In [4]:
df

,reviews.text,reviews.title
0,This product so far has not disappointed. My c...,Kindle
1,great for beginner or experienced person. Boug...,very fast
2,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.
3,I've had my Fire HD 8 two weeks now and I love...,Good!!!
4,I bought this for my grand daughter when she c...,Fantastic Tablet for kids
...,...,...
4995,Amazing tablet for the price i love it is real...,Great tablet
4996,I love my fire... The games and the pictures g...,fire
4997,Great tablet for kids but the screen isn't as ...,Fire
4998,Great product easy to use. Having different co...,kindle


Lower Casing:

In [5]:
df["reviews.text.lower"] = df["reviews.text"].str.lower()
df["reviews.title.lower"]=df["reviews.title"].str.lower()
df.head()

<ipython-input-5-741b18d3e0a5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["reviews.text.lower"] = df["reviews.text"].str.lower()
<ipython-input-5-741b18d3e0a5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["reviews.title.lower"]=df["reviews.title"].str.lower()


,reviews.text,reviews.title,reviews.text.lower,reviews.title.lower
0,This product so far has not disappointed. My c...,Kindle,this product so far has not disappointed. my c...,kindle
1,great for beginner or experienced person. Boug...,very fast,great for beginner or experienced person. boug...,very fast
2,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,inexpensive tablet for him to use and learn on...,beginner tablet for our 9 year old son.
3,I've had my Fire HD 8 two weeks now and I love...,Good!!!,i've had my fire hd 8 two weeks now and i love...,good!!!
4,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,i bought this for my grand daughter when she c...,fantastic tablet for kids


Removal of Punctuations:

In [6]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
df["reviews.text.wo_punc"] = df["reviews.text"].apply(lambda text: remove_punctuation(text))
df["reviews.title.wo_punc"]=df["reviews.title"].apply(lambda text: remove_punctuation(text))

df.head()

<ipython-input-6-98db9287eb45>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["reviews.text.wo_punc"] = df["reviews.text"].apply(lambda text: remove_punctuation(text))
<ipython-input-6-98db9287eb45>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["reviews.title.wo_punc"]=df["reviews.title"].apply(lambda text: remove_punctuation(text))


,reviews.text,reviews.title,reviews.text.lower,reviews.title.lower,reviews.text.wo_punc,reviews.title.wo_punc
0,This product so far has not disappointed. My c...,Kindle,this product so far has not disappointed. my c...,kindle,This product so far has not disappointed My ch...,Kindle
1,great for beginner or experienced person. Boug...,very fast,great for beginner or experienced person. boug...,very fast,great for beginner or experienced person Bough...,very fast
2,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,inexpensive tablet for him to use and learn on...,beginner tablet for our 9 year old son.,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son
3,I've had my Fire HD 8 two weeks now and I love...,Good!!!,i've had my fire hd 8 two weeks now and i love...,good!!!,Ive had my Fire HD 8 two weeks now and I love ...,Good
4,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,i bought this for my grand daughter when she c...,fantastic tablet for kids,I bought this for my grand daughter when she c...,Fantastic Tablet for kids


Removal of stopwords:

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [9]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["reviews.text.wo_stop"] = df["reviews.text.wo_punc"].apply(lambda text: remove_stopwords(text))
df["reviews.title.wo_stop"] = df["reviews.title.wo_punc"].apply(lambda text: remove_stopwords(text))
df.head()

<ipython-input-9-656bb0899144>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["reviews.text.wo_stop"] = df["reviews.text.wo_punc"].apply(lambda text: remove_stopwords(text))
<ipython-input-9-656bb0899144>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["reviews.title.wo_stop"] = df["reviews.title.wo_punc"].apply(lambda text: remove_stopwords(text))


,reviews.text,reviews.title,reviews.text.lower,reviews.title.lower,reviews.text.wo_punc,reviews.title.wo_punc,reviews.text.wo_stop,reviews.title.wo_stop
0,This product so far has not disappointed. My c...,Kindle,this product so far has not disappointed. my c...,kindle,This product so far has not disappointed My ch...,Kindle,This product far disappointed My children love...,Kindle
1,great for beginner or experienced person. Boug...,very fast,great for beginner or experienced person. boug...,very fast,great for beginner or experienced person Bough...,very fast,great beginner experienced person Bought gift ...,fast
2,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,inexpensive tablet for him to use and learn on...,beginner tablet for our 9 year old son.,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son,Inexpensive tablet use learn step NABI He thri...,Beginner tablet 9 year old son
3,I've had my Fire HD 8 two weeks now and I love...,Good!!!,i've had my fire hd 8 two weeks now and i love...,good!!!,Ive had my Fire HD 8 two weeks now and I love ...,Good,Ive Fire HD 8 two weeks I love This tablet gre...,Good
4,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,i bought this for my grand daughter when she c...,fantastic tablet for kids,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,I bought grand daughter comes visit I set user...,Fantastic Tablet kids


Removal of Frequent words

In [10]:
from collections import Counter
cnt = Counter()
for text in df["reviews.text.wo_stop"].values:
    for word in text.split():
        cnt[word] += 1

cnt.most_common(10)

[('I', 4381),
 ('tablet', 2015),
 ('great', 1248),
 ('use', 1113),
 ('price', 926),
 ('The', 887),
 ('good', 743),
 ('This', 741),
 ('Kindle', 708),
 ('one', 686)]

In [11]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

df["reviews.text.wo_stopfreq"] = df["reviews.text.wo_stop"].apply(lambda text: remove_freqwords(text))
df.head()

<ipython-input-11-f27f8bc4ce05>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["reviews.text.wo_stopfreq"] = df["reviews.text.wo_stop"].apply(lambda text: remove_freqwords(text))


,reviews.text,reviews.title,reviews.text.lower,reviews.title.lower,reviews.text.wo_punc,reviews.title.wo_punc,reviews.text.wo_stop,reviews.title.wo_stop,reviews.text.wo_stopfreq
0,This product so far has not disappointed. My c...,Kindle,this product so far has not disappointed. my c...,kindle,This product so far has not disappointed My ch...,Kindle,This product far disappointed My children love...,Kindle,product far disappointed My children love like...
1,great for beginner or experienced person. Boug...,very fast,great for beginner or experienced person. boug...,very fast,great for beginner or experienced person Bough...,very fast,great beginner experienced person Bought gift ...,fast,beginner experienced person Bought gift loves
2,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,inexpensive tablet for him to use and learn on...,beginner tablet for our 9 year old son.,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son,Inexpensive tablet use learn step NABI He thri...,Beginner tablet 9 year old son,Inexpensive learn step NABI He thrilled learn ...
3,I've had my Fire HD 8 two weeks now and I love...,Good!!!,i've had my fire hd 8 two weeks now and i love...,good!!!,Ive had my Fire HD 8 two weeks now and I love ...,Good,Ive Fire HD 8 two weeks I love This tablet gre...,Good,Ive Fire HD 8 two weeks love valueWe Prime Mem...
4,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,i bought this for my grand daughter when she c...,fantastic tablet for kids,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,I bought grand daughter comes visit I set user...,Fantastic Tablet kids,bought grand daughter comes visit set user ent...


In [12]:
from collections import Counter
cnt = Counter()
for text in df["reviews.title.wo_stop"].values:
    for word in text.split():
        cnt[word] += 1

cnt.most_common(2)

[('Great', 1501), ('tablet', 1066)]

In [13]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(2)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

df["reviews.title.wo_stopfreq"] = df["reviews.title.wo_stop"].apply(lambda text: remove_freqwords(text))
df.head()

<ipython-input-13-61e69abfddd9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["reviews.title.wo_stopfreq"] = df["reviews.title.wo_stop"].apply(lambda text: remove_freqwords(text))


,reviews.text,reviews.title,reviews.text.lower,reviews.title.lower,reviews.text.wo_punc,reviews.title.wo_punc,reviews.text.wo_stop,reviews.title.wo_stop,reviews.text.wo_stopfreq,reviews.title.wo_stopfreq
0,This product so far has not disappointed. My c...,Kindle,this product so far has not disappointed. my c...,kindle,This product so far has not disappointed My ch...,Kindle,This product far disappointed My children love...,Kindle,product far disappointed My children love like...,Kindle
1,great for beginner or experienced person. Boug...,very fast,great for beginner or experienced person. boug...,very fast,great for beginner or experienced person Bough...,very fast,great beginner experienced person Bought gift ...,fast,beginner experienced person Bought gift loves,fast
2,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,inexpensive tablet for him to use and learn on...,beginner tablet for our 9 year old son.,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son,Inexpensive tablet use learn step NABI He thri...,Beginner tablet 9 year old son,Inexpensive learn step NABI He thrilled learn ...,Beginner 9 year old son
3,I've had my Fire HD 8 two weeks now and I love...,Good!!!,i've had my fire hd 8 two weeks now and i love...,good!!!,Ive had my Fire HD 8 two weeks now and I love ...,Good,Ive Fire HD 8 two weeks I love This tablet gre...,Good,Ive Fire HD 8 two weeks love valueWe Prime Mem...,Good
4,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,i bought this for my grand daughter when she c...,fantastic tablet for kids,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,I bought grand daughter comes visit I set user...,Fantastic Tablet kids,bought grand daughter comes visit set user ent...,Fantastic Tablet kids


Removal of Rare words

In [14]:
# Drop the two columns which are no more needed
df.drop(["reviews.text.lower","reviews.title.lower","reviews.text.wo_punc","reviews.title.wo_punc","reviews.text.wo_stop","reviews.title.wo_stop"], axis=1, inplace=True)

n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

df["reviews.text.wo_stopfreqrare"] = df["reviews.text.wo_stopfreq"].apply(lambda text: remove_rarewords(text))
df["reviews.title.wo_stopfreqrare"] = df["reviews.title.wo_stopfreq"].apply(lambda text: remove_rarewords(text))
df.head()

<ipython-input-14-535564c8b018>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(["reviews.text.lower","reviews.title.lower","reviews.text.wo_punc","reviews.title.wo_punc","reviews.text.wo_stop","reviews.title.wo_stop"], axis=1, inplace=True)
<ipython-input-14-535564c8b018>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["reviews.text.wo_stopfreqrare"] = df["reviews.text.wo_stopfreq"].apply(lambda text: remove_rarewords(text))
<ipython-input-14-535564c8b018>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

,reviews.text,reviews.title,reviews.text.wo_stopfreq,reviews.title.wo_stopfreq,reviews.text.wo_stopfreqrare,reviews.title.wo_stopfreqrare
0,This product so far has not disappointed. My c...,Kindle,product far disappointed My children love like...,Kindle,product far disappointed My children love like...,Kindle
1,great for beginner or experienced person. Boug...,very fast,beginner experienced person Bought gift loves,fast,beginner experienced person Bought gift loves,fast
2,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,Inexpensive learn step NABI He thrilled learn ...,Beginner 9 year old son,Inexpensive learn step NABI He thrilled learn ...,Beginner 9 year old son
3,I've had my Fire HD 8 two weeks now and I love...,Good!!!,Ive Fire HD 8 two weeks love valueWe Prime Mem...,Good,Ive Fire HD 8 two weeks love valueWe Prime Mem...,Good
4,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,bought grand daughter comes visit set user ent...,Fantastic Tablet kids,bought grand daughter comes visit set user ent...,Fantastic Tablet kids


Stemming

In [15]:
from nltk.stem.porter import PorterStemmer

# Drop the two columns
df.drop(["reviews.text.wo_stopfreq","reviews.title.wo_stopfreq","reviews.text.wo_stopfreqrare","reviews.title.wo_stopfreqrare"], axis=1, inplace=True)

stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

df["reviews.text_stemmed"] = df["reviews.text"].apply(lambda text: stem_words(text))
df["reviews.title_stemmed"] = df["reviews.title"].apply(lambda text: stem_words(text))
df.head()

<ipython-input-15-97c8a5cd9573>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(["reviews.text.wo_stopfreq","reviews.title.wo_stopfreq","reviews.text.wo_stopfreqrare","reviews.title.wo_stopfreqrare"], axis=1, inplace=True)
<ipython-input-15-97c8a5cd9573>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["reviews.text_stemmed"] = df["reviews.text"].apply(lambda text: stem_words(text))
<ipython-input-15-97c8a5cd9573>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

,reviews.text,reviews.title,reviews.text_stemmed,reviews.title_stemmed
0,This product so far has not disappointed. My c...,Kindle,thi product so far ha not disappointed. my chi...,kindl
1,great for beginner or experienced person. Boug...,very fast,great for beginn or experienc person. bought a...,veri fast
2,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,"inexpens tablet for him to use and learn on, s...",beginn tablet for our 9 year old son.
3,I've had my Fire HD 8 two weeks now and I love...,Good!!!,i'v had my fire hd 8 two week now and i love i...,good!!!
4,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,i bought thi for my grand daughter when she co...,fantast tablet for kid


In [16]:
from nltk.stem.snowball import SnowballStemmer
SnowballStemmer.languages

('arabic',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'hungarian',
 'italian',
 'norwegian',
 'porter',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish')

Lemmatization

In [17]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df["reviews.text_lemmatized"] = df["reviews.text"].apply(lambda text: lemmatize_words(text))
df["reviews.title_lemmatized"] = df["reviews.title"].apply(lambda text: lemmatize_words(text))
df.head()

<ipython-input-18-d61600fba7e0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["reviews.text_lemmatized"] = df["reviews.text"].apply(lambda text: lemmatize_words(text))
<ipython-input-18-d61600fba7e0>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["reviews.title_lemmatized"] = df["reviews.title"].apply(lambda text: lemmatize_words(text))


,reviews.text,reviews.title,reviews.text_stemmed,reviews.title_stemmed,reviews.text_lemmatized,reviews.title_lemmatized
0,This product so far has not disappointed. My c...,Kindle,thi product so far ha not disappointed. my chi...,kindl,This product so far ha not disappointed. My ch...,Kindle
1,great for beginner or experienced person. Boug...,very fast,great for beginn or experienc person. bought a...,veri fast,great for beginner or experienced person. Boug...,very fast
2,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,"inexpens tablet for him to use and learn on, s...",beginn tablet for our 9 year old son.,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.
3,I've had my Fire HD 8 two weeks now and I love...,Good!!!,i'v had my fire hd 8 two week now and i love i...,good!!!,I've had my Fire HD 8 two week now and I love ...,Good!!!
4,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,i bought thi for my grand daughter when she co...,fantast tablet for kid,I bought this for my grand daughter when she c...,Fantastic Tablet for kid
